# PyTorch with Kubeflow Fairing 

In this notebook we will walk through training a character recongition model using the MNIST dataset on Pytorch. 
We will then show you how to use Kubeflow Fairing to run the same training job on both Kubeflow and CMLE

In [1]:
#you can skip this step if you have already installed the necessary dependencies
!pip install -U -r requirements.txt

     |██████████████                  | 325.1 MB 119.9 MB/s eta 0:00:04| 254.5 MB 65.2 MB/s eta 0:00:08     |████████████▏                   | 284.4 MB 65.2 MB/s eta 0:00:08

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |█████████████████▍              | 406.2 MB 119.9 MB/s eta 0:00:03

     |█████████████████▋              | 412.2 MB 38.7 MB/s eta 0:00:09

     |█████████████████▉              | 418.2 MB 38.7 MB/s eta 0:00:09

...

     |███████████████████████████████▍| 734.0 MB 128.9 MB/s eta 0:00:01

     |███████████████████████████████▋| 739.6 MB 128.9 MB/s eta 0:00:01

     |███████████████████████████████▉| 745.7 MB 128.9 MB/s eta 0:00:01

     |████████████████████████████████| 748.8 MB 128.9 MB/s 


     |████████████████████████████████| 5.9 MB 66.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 60.3 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [8]:
%%writefile train_model.py
import argparse
import os
import subprocess
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
# For mac users you may get hit with this bug https://github.com/pytorch/pytorch/issues/20030
# temporary solution is "brew install libomp"

Overwriting train_model.py


## PyTorch Model Defintion

Setup a Convolution Nueral network using Pytorch!

In [9]:
%%writefile train_model.py -a
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

Appending to train_model.py


## PyTorch Training and Test Functions
A simple training function that batches the data set. 

In [10]:
%%writefile train_model.py -a
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0 and batch_idx>0:
            print('Train Epoch: {}\t[{}/{}\t({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))

Appending to train_model.py


In [11]:
%%writefile train_model.py -a
def test(model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(
              output, target, size_average=False).item()  # sum up batch loss
            pred = output.max(
              1, keepdim=True)[1]  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(test_loader.dataset),
          100. * correct / len(test_loader.dataset)))

Appending to train_model.py


In [12]:
%%writefile train_model.py -a
def nkTrain(batch_size=64, epochs=1, log_interval=100, lr=0.01, model_dir=None, momentum=0.5, 
                       no_cuda=False, seed=1, test_batch_size=1000):

    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(seed)
    device = torch.device('cuda' if use_cuda else 'cpu')
    print("Using {} for training.".format(device))

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
      datasets.MNIST(
          'data',
          train=True,
          download=True,
          transform=transforms.Compose([
              transforms.ToTensor(),
              # Normalize a tensor image with mean and standard deviation
              transforms.Normalize(mean=(0.1307,), std=(0.3081,))
          ])),
      batch_size=batch_size,
      shuffle=True,
      **kwargs)
    test_loader = torch.utils.data.DataLoader(
      datasets.MNIST(
          'data',
          train=False,
          transform=transforms.Compose([
              transforms.ToTensor(),
              # Normalize a tensor image with mean and standard deviation              
              transforms.Normalize(mean=(0.1307,), std=(0.3081,))
          ])),
      batch_size=test_batch_size,
      shuffle=True,
      **kwargs)

    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(1, epochs + 1):
        start_time = time.time()
        train(model, device, train_loader, optimizer, epoch, log_interval)
        print("Time taken for epoch #{}: {:.2f}s".format(epoch, time.time()-start_time))
        test(model, device, test_loader, epoch)

    if model_dir:
        model_file_name = 'torch.model'
        tmp_model_file = os.path.join('/tmp', model_file_name)
        torch.save(model.state_dict(), tmp_model_file)
        subprocess.check_call([
            'gsutil', 'cp', tmp_model_file,
            os.path.join(model_dir, model_file_name)])

Appending to train_model.py


## Training locally

In [7]:
nkTrain()

Using cpu for training.


100.1%

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


113.5%

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


...

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!
Train Epoch: 1	[6400/60000	(11%)]	Loss: 1.929761
Train Epoch: 1	[12800/60000	(21%)]	Loss: 1.327502
Train Epoch: 1	[19200/60000	(32%)]	Loss: 0.846595
Train Epoch: 1	[25600/60000	(43%)]	Loss: 0.674760
Train Epoch: 1	[32000/60000	(53%)]	Loss: 0.442683
Train Epoch: 1	[38400/60000	(64%)]	Loss: 0.704966
Train Epoch: 1	[44800/60000	(75%)]	Loss: 0.470975
Train Epoch: 1	[51200/60000	(85%)]	Loss: 0.810192
Train Epoch: 1	[57600/60000	(96%)]	Loss: 0.412998
Time taken for epoch #1: 18.48s


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.2063, Accuracy: 9387/10000 (94%)



## Build Docker Images and train using the container.

In this block we set some Docker config. Fairing will use this information to package up the `train_and_test` function i

In [13]:
!nkode create:image -d "data/MNIST"

hello auto from ./src/commands/hello.js
sh /usr/lib/node_modules/nkode/scripts/remoteTrain/remote_train.sh  auto data/MNIST tensorflow/tensorflow:1.15.0-py3
auto data/MNIST tensorflow/tensorflow:1.15.0-py3
[I 201022 20:59:11 utils:320] IMDS ENDPOINT: http://169.254.169.254/
[W 201022 20:59:11 function:49] The FunctionPreProcessor is optimized for using in a notebook or IPython environment. For it to work, the python version should be same for both local python and the python in the docker. Please look at alternatives like BasePreprocessor or FullNotebookPreprocessor.
[W 201022 20:59:11 tasks:62] Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
[I 201022 20:59:11 tasks:66] Building the docker image.
[I 201022 20:59:11 cluster:46] Building image using cluster builder.
[W 201022 20:59:11 base:94] /usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 201022 20:59:11 base:107] Creating docker co